# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
#g_key


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
#Reading data file
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_df.head()

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,taihe,30.87,105.38,56.84,80.0,67.0,3.13,CN,1.603228e+09
1,mitu,1.20,-70.17,81.05,90.0,99.0,2.01,CO,1.603228e+09
2,mawlaik,23.63,94.42,72.81,94.0,90.0,1.10,MM,1.603228e+09
3,reyes,-14.32,-67.38,91.40,70.0,75.0,11.41,BO,1.603228e+09
4,vesyegonsk,58.66,37.26,30.11,96.0,16.0,8.41,RU,1.603228e+09


In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
places = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [26]:
# Plotinf the Heatmap
fig = gmaps.figure(center=(20.0, -5), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(places, weights = humidity, dissipating=False, max_intensity=100, point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
max_intensity

100.0

In [28]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
selected_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 5) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 75) & (weather_df["Max Temp"] <= 85)].dropna()

selected_city_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,miandrivazo,-19.52,45.47,81.52,48.0,0.0,3.15,MG,1.603228e+09
122,shakawe,-18.37,21.85,75.51,18.0,0.0,1.86,BW,1.603228e+09
256,doka,13.52,35.77,79.47,35.0,0.0,0.16,SD,1.603228e+09
283,sharjah,25.36,55.40,80.60,56.0,0.0,4.70,AE,1.603228e+09
297,hostotipaquillo,21.07,-104.07,84.99,45.0,0.0,1.99,MX,1.603228e+09
308,aswan,24.09,32.91,79.66,25.0,0.0,3.87,EG,1.603228e+09
432,kaura namoda,12.59,6.58,82.49,30.0,0.0,2.62,NG,1.603228e+09
460,qandala,11.47,49.87,83.97,39.0,0.0,3.15,SO,1.603228e+09
542,kolokani,13.57,-8.03,82.94,24.0,0.0,2.59,ML,1.603228e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# Create a hotel_df
hotel_df = selected_city_df.loc[:,["City Name","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [30]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # looking for city, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City Name"]
    
    # add "location" to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    print(requests.get(base_url, params=params))
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print("-------------------------")
        print(f"City name: {city}")
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
 

Retrieving Results for Index 30: miandrivazo.
<Response [200]>
-------------------------
City name: miandrivazo
Closest hotel in miandrivazo is Princesse Tsiribihina.
Retrieving Results for Index 122: shakawe.
<Response [200]>
-------------------------
City name: shakawe
Closest hotel in shakawe is Drotsky's Cabins.
Retrieving Results for Index 256: doka.
<Response [200]>
-------------------------
City name: doka
Missing field/result... skipping.
Retrieving Results for Index 283: sharjah.
<Response [200]>
-------------------------
City name: sharjah
Closest hotel in sharjah is Sheraton Sharjah Beach Resort & Spa.
Retrieving Results for Index 297: hostotipaquillo.
<Response [200]>
-------------------------
City name: hostotipaquillo
Closest hotel in hostotipaquillo is Hotel Hosto Real.
Retrieving Results for Index 308: aswan.
<Response [200]>
-------------------------
City name: aswan
Closest hotel in aswan is Hotel Sofitel Legend Old Cataract Aswan.
Retrieving Results for Index 432: ka

In [31]:
# Display the hotel dataframe
hotel_df

,City Name,Country,Lat,Lng,Hotel Name
30,miandrivazo,MG,-19.52,45.47,Princesse Tsiribihina
122,shakawe,BW,-18.37,21.85,Drotsky's Cabins
256,doka,SD,13.52,35.77,
283,sharjah,AE,25.36,55.40,Sheraton Sharjah Beach Resort & Spa
297,hostotipaquillo,MX,21.07,-104.07,Hotel Hosto Real
308,aswan,EG,24.09,32.91,Hotel Sofitel Legend Old Cataract Aswan
432,kaura namoda,NG,12.59,6.58,Moonlight Hotel
460,qandala,SO,11.47,49.87,
542,kolokani,ML,13.57,-8.03,Maimouna sylla


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>